In [19]:
import pandas as pd
import numpy as np
import json
import seaborn as sns

In [23]:
catalog = pd.read_csv('data/catalog.gz')

In [24]:
catalog.head()

,pid,current_price,original_price,category,sub_category,sub_sub_category
0,c9fe04e5097c087b6b6eeccc3adc4a142e14aa46,699.0,NaN,c1bd5fd999bd577743936898ada96496b547af3c,f08770a96fb546673053ab799f5ea9cada06c06a,2d2c44a2d8f18a6271f0e8057313af68a46d0f24
1,e00988f42f7fc5f14ec6a0e7905789110f23c5de,150.0,NaN,37b2661cc181c987f68770e43990152026920ba5,11e623a37e87cf7995c466723ec99688d55cae8c,11e623a37e87cf7995c466723ec99688d55cae8c
2,1e2e7c0d4082295728c1684a702cb43e1d332d3e,549.9,1049.9,c37df0ae71b97699a478def3001a3516a905a51d,f08770a96fb546673053ab799f5ea9cada06c06a,d6605426aaa703bf19874c0caac79a661b73de33
3,a279dd2284eb57533ca417c258ede0a0526a6f6e,0.0,0.0,c1bd5fd999bd577743936898ada96496b547af3c,6d9d48ae11ee1909235f31e9bfe5d36aa1462cb3,9a2e3cb56f6a1756fd35a4fd70172a67ecf13639
4,405b0362ae6ec149700164811b0e7773c8300e9d,79.9,0.0,c37df0ae71b97699a478def3001a3516a905a51d,78af0aac89e0f15a6c9fc70b5bff79d98c6dcc43,2541a58f702844477aab540e5df7b859a1e3d5de


In [35]:
with open('data/test', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        json_records.append(record)

In [36]:
json_records

[{u'event_type': u'pageview',
  u'page_type': u'cart',
  u'source': nan,
  u'timestamp': u'2016-01-01',
  u'uid': u'54cfe1703c536e98bf02c7c4cd5b4e280f07d74c'},
 {u'event_type': u'pageview',
  u'page_type': u'checkout',
  u'source': nan,
  u'timestamp': u'2016-01-02',
  u'uid': u'c808d5ecb73bee00a9730064b7652bec10778f20'},
 {u'event_type': u'pageview',
  u'page_type': u'checkout',
  u'source': nan,
  u'timestamp': u'2016-01-02',
  u'uid': u'7ba0cbaaf2000d440b474ff9182383c5525d0d56'},
 {u'event_type': u'pageview',
  u'page_type': u'confirmation',
  u'source': nan,
  u'timestamp': u'2016-01-02',
  u'uid': u'3b2790a1f746e59298c8c277e3ec2279343ce52d'},
 {u'event_type': u'pageview',
  u'page_type': u'checkout',
  u'source': nan,
  u'timestamp': u'2016-01-02',
  u'uid': u'3b2790a1f746e59298c8c277e3ec2279343ce52d'},
 {u'event_type': u'pageview',
  u'page_type': u'product',
  u'productId': u'82bf3f74e5bc4b9f612f9d7addc5967eba70b758',
  u'source': nan,
  u'timestamp': u'2016-07-31',
  u'uid': u'

In [37]:
pd.DataFrame(json_records)

,date,event_type,page_type,productId,products,source,timestamp,uid
0,NaN,pageview,cart,NaN,NaN,NaN,2016-01-01,54cfe1703c536e98bf02c7c4cd5b4e280f07d74c
1,NaN,pageview,checkout,NaN,NaN,NaN,2016-01-02,c808d5ecb73bee00a9730064b7652bec10778f20
2,NaN,pageview,checkout,NaN,NaN,NaN,2016-01-02,7ba0cbaaf2000d440b474ff9182383c5525d0d56
3,NaN,pageview,confirmation,NaN,NaN,NaN,2016-01-02,3b2790a1f746e59298c8c277e3ec2279343ce52d
4,NaN,pageview,checkout,NaN,NaN,NaN,2016-01-02,3b2790a1f746e59298c8c277e3ec2279343ce52d
5,NaN,pageview,product,82bf3f74e5bc4b9f612f9d7addc5967eba70b758,NaN,NaN,2016-07-31,8111a0cbcadd0c302af93a389db63ab218c2b88d
6,NaN,pageview,product,2a9dd6a8cda2a1797ce60cf4207ef8c2510afac1,NaN,NaN,2016-07-31,8111a0cbcadd0c302af93a389db63ab218c2b88d
7,NaN,pageview,product,6f946e26bbbc095620c42c4453cbf8df7ffca1a0,NaN,NaN,2016-07-31,4efa3f9a8b8fbe208f7da61c6461a57ca6fb7507
8,NaN,pageview,product,2e4f4690a3a351effef7cfd3f42711e90716ace3,NaN,NaN,2016-07-31,a33698c16c0e73bb2118e4924ee646f954cdd6fc
9,NaN,pageview,product,0ebaeb0ce43ad6adf0e9e231419beb89ea177985,NaN,NaN,2016-07-31,a33698c16c0e73bb2118e4924ee646f954cdd6fc


In [39]:
oi = [{'col1':'oi', 'col2':'ola'},{'col1':'oi2','col2':'ola2'}]

In [42]:
pd.DataFrame(oi)

,col1,col2
0,oi,ola
1,oi2,ola2


In [38]:
pd.read_json?